In [7]:
import pandas as pd
import numpy as np
from rapidfuzz import process
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', None)

# Load data
deliveries = pd.read_csv('all_matches_updated.csv')
df = deliveries.copy()

# Data preparation
df = df.rename(columns={'striker': 'batsman'})
df = df.rename(columns={'runs_off_bat': 'runs_of_bat'})
df['innings'] = df['innings'].astype(int)
df['wides'] = df['wides'].fillna(0)
df['noballs'] = df['noballs'].fillna(0)
df['total_runs'] = df['runs_of_bat'] + df['wides'] + df['noballs']

def enhanced_player_analysis_with_overall_stats(df, venue_name, input_players, threshold=80):
    """Enhanced player analysis with comprehensive overall statistics and abbreviations"""
    
    venue_batsmen = df[df['venue'] == venue_name]['batsman'].unique().tolist()
    
    if not venue_batsmen:
        print(f"❌ No data found for venue: {venue_name}")
        return
    
    print(f"🏟️ PLAYER PERFORMANCE ANALYSIS")
    print(f"📍 Venue: {venue_name}")
    print("=" * 120)
    
    # Enhanced fuzzy search with RapidFuzz - FIXED: unpack 3 values instead of 2
    matched_players = {}
    exact_matches = []
    
    for player in input_players:
        matches = process.extract(player, venue_batsmen, limit=5)
        # FIX: RapidFuzz returns (name, score, index) - unpack all 3 values
        good_matches = [(name, score, index) for name, score, index in matches if score >= threshold]
        
        if good_matches:
            best_match = good_matches[0][0]  # Still get the name (first element)
            exact_matches.append(best_match)
            matched_players[player] = {
                'found': True,
                'match': best_match,
                'confidence': good_matches[0][1]  # Still get the score (second element)
            }
        else:
            matched_players[player] = {
                'found': False,
                'suggestions': matches[:2]
            }
    
    # Display search results
    print(f"\n🔍 PLAYER SEARCH RESULTS:")
    print("-" * 50)
    found_count = 0
    for input_player, result in matched_players.items():
        if result['found']:
            confidence_emoji = "🎯" if result['confidence'] == 100 else "✅"
            print(f"{confidence_emoji} {input_player} → {result['match']} ({result['confidence']}%)")
            found_count += 1
        else:
            print(f"❌ {input_player} → Not found")
            # FIX: Handle the 3-tuple format in suggestions too
            suggestions = ", ".join([f"{name} ({score}%)" for name, score, index in result['suggestions']])
            print(f"   💡 Try: {suggestions}")
    
    if found_count == 0:
        print("\n⚠️ No players found at this venue.")
        return
    
    print(f"\n📊 Found {found_count} out of {len(input_players)} players")
    print("=" * 120)
    
    # Analyze each found player with comprehensive metrics
    all_summaries = []
    
    for player in exact_matches:
        # Get player data at this venue
        venue_player_data = df[(df['venue'] == venue_name) & (df['batsman'] == player)]
        
        # Get player data across ALL venues for overall statistics
        all_venues_player_data = df[df['batsman'] == player]
        
        if venue_player_data.empty:
            continue
        
        # VENUE-SPECIFIC CALCULATIONS
        venue_total_matches = venue_player_data['match_id'].nunique()
        venue_total_runs = venue_player_data['runs_of_bat'].sum()
        venue_total_balls = len(venue_player_data)
        venue_total_dismissals = venue_player_data[venue_player_data['player_dismissed'] == player].shape[0]
        
        # Venue boundaries
        venue_fours = (venue_player_data['runs_of_bat'] == 4).sum()
        venue_sixes = (venue_player_data['runs_of_bat'] == 6).sum()
        venue_boundaries = venue_fours + venue_sixes
        
        # OVERALL (ALL VENUES) CALCULATIONS
        overall_total_matches = all_venues_player_data['match_id'].nunique()
        overall_total_runs = all_venues_player_data['runs_of_bat'].sum()
        overall_total_balls = len(all_venues_player_data)
        overall_total_dismissals = all_venues_player_data[all_venues_player_data['player_dismissed'] == player].shape[0]
        
        # Overall boundaries
        overall_fours = (all_venues_player_data['runs_of_bat'] == 4).sum()
        overall_sixes = (all_venues_player_data['runs_of_bat'] == 6).sum()
        overall_boundaries = overall_fours + overall_sixes
        
        # Calculate averages
        venue_avg_per_match = venue_total_runs / venue_total_matches if venue_total_matches > 0 else 0
        overall_avg_per_match = overall_total_runs / overall_total_matches if overall_total_matches > 0 else 0
        
        # VENUE innings-wise analysis
        venue_innings_data = venue_player_data.groupby(['match_id', 'innings'])['runs_of_bat'].sum().reset_index()
        venue_inn1_data = venue_innings_data[venue_innings_data['innings'] == 1]
        venue_inn1_avg = venue_inn1_data['runs_of_bat'].mean() if len(venue_inn1_data) > 0 else 0
        venue_inn2_data = venue_innings_data[venue_innings_data['innings'] == 2]
        venue_inn2_avg = venue_inn2_data['runs_of_bat'].mean() if len(venue_inn2_data) > 0 else 0
        
        # OVERALL innings-wise analysis
        overall_innings_data = all_venues_player_data.groupby(['match_id', 'innings'])['runs_of_bat'].sum().reset_index()
        overall_inn1_data = overall_innings_data[overall_innings_data['innings'] == 1]
        overall_inn1_avg = overall_inn1_data['runs_of_bat'].mean() if len(overall_inn1_data) > 0 else 0
        overall_inn2_data = overall_innings_data[overall_innings_data['innings'] == 2]
        overall_inn2_avg = overall_inn2_data['runs_of_bat'].mean() if len(overall_inn2_data) > 0 else 0
        
        # Performance metrics
        venue_strike_rate = (venue_total_runs / venue_total_balls * 100) if venue_total_balls > 0 else 0
        overall_strike_rate = (overall_total_runs / overall_total_balls * 100) if overall_total_balls > 0 else 0
        venue_highest_score = venue_innings_data['runs_of_bat'].max() if not venue_innings_data.empty else 0
        overall_highest_score = overall_innings_data['runs_of_bat'].max() if not overall_innings_data.empty else 0
        
        # Calculate BPD and BPB
        venue_bpd = round(venue_total_balls / venue_total_dismissals, 2) if venue_total_dismissals > 0 else 0
        venue_bpb = round(venue_total_balls / venue_boundaries, 2) if venue_boundaries > 0 else 0
        overall_bpd = round(overall_total_balls / overall_total_dismissals, 2) if overall_total_dismissals > 0 else 0
        overall_bpb = round(overall_total_balls / overall_boundaries, 2) if overall_boundaries > 0 else 0
        
        # Milestones
        venue_thirties = (venue_innings_data['runs_of_bat'] >= 30).sum()
        venue_fifties = (venue_innings_data['runs_of_bat'] >= 50).sum()
        venue_hundreds = (venue_innings_data['runs_of_bat'] >= 100).sum()
        
        overall_thirties = (overall_innings_data['runs_of_bat'] >= 30).sum()
        overall_fifties = (overall_innings_data['runs_of_bat'] >= 50).sum()
        overall_hundreds = (overall_innings_data['runs_of_bat'] >= 100).sum()
        
        # Display individual player summary
        print(f"\n🏏 {player}")
        print("-" * 80)
        print(f"📈 Performance Comparison:")
        print(f"   • Overall: {overall_total_matches} Mat, {overall_total_runs} Runs, Avg: {overall_avg_per_match:.1f}, SR: {overall_strike_rate:.1f}")
        print(f"   • At Venue: {venue_total_matches} Mat, {venue_total_runs} Runs, Avg: {venue_avg_per_match:.1f}, SR: {venue_strike_rate:.1f}")
        print(f"   • Overall BPD: {overall_bpd} | Venue BPD: {venue_bpd}")
        print(f"   • Overall Milestones: {overall_thirties}|{overall_fifties}|{overall_hundreds} | Venue: {venue_thirties}|{venue_fifties}|{venue_hundreds}")
        
        # Store for comparison table with ALL requested columns
        all_summaries.append({
            'Player': player,
            'O_Mat': overall_total_matches,           # Overall Matches
            'O_Runs': overall_total_runs,             # Overall Runs  
            'O_Avg': round(overall_avg_per_match, 1), # Overall Average
            'O_1st': round(overall_inn1_avg, 1),      # Overall 1st Inn Avg
            'O_2nd': round(overall_inn2_avg, 1),      # Overall 2nd Inn Avg
            'O_BPD': overall_bpd,                     # Overall BPD
            'O_Mile': f"{overall_thirties}|{overall_fifties}|{overall_hundreds}",  # Overall Milestones
            'V_Mat': venue_total_matches,             # Venue Matches
            'V_Runs': venue_total_runs,               # Venue Runs
            'V_Avg': round(venue_avg_per_match, 1),   # Venue Average
            'V_SR': round(venue_strike_rate, 1),      # Venue Strike Rate
            'V_HS': venue_highest_score,              # Venue Highest Score
            'V_Bnd': venue_boundaries,                # Venue Boundaries
            'V_1st': round(venue_inn1_avg, 1),        # Venue 1st Inn Avg
            'V_2nd': round(venue_inn2_avg, 1),        # Venue 2nd Inn Avg
            'V_BPD': venue_bpd,                       # Venue BPD
            'V_BPB': venue_bpb,                       # Venue BPB
            'V_Mile': f"{venue_thirties}|{venue_fifties}|{venue_hundreds}"  # Venue Milestones
        })
    
    # Display comparison table with abbreviated column names
    if len(all_summaries) > 1:
        comparison_df = pd.DataFrame(all_summaries)
        comparison_df = comparison_df.sort_values('V_Avg', ascending=False)
        comparison_df.index = range(1, len(comparison_df) + 1)
        
        print(f"\n📋 COMPREHENSIVE PLAYER COMPARISON AT {venue_name.upper()}")
        print("=" * 120)
        print("Legend: O_ = Overall (All Venues), V_ = Venue Specific")
        print("Mat=Matches, Avg=Average, SR=Strike Rate, HS=Highest Score, Bnd=Boundaries")
        print("BPD=Balls Per Dismissal, BPB=Balls Per Boundary, Mile=Milestones (30s|50s|100s)")
        print("-" * 120)
        
        from IPython.display import display
        display(comparison_df)
        
        # Enhanced insights
        best_overall_avg = comparison_df.loc[comparison_df['O_Avg'].idxmax()]
        best_venue_avg = comparison_df.loc[comparison_df['V_Avg'].idxmax()]
        most_overall_runs = comparison_df.loc[comparison_df['O_Runs'].idxmax()]
        most_venue_runs = comparison_df.loc[comparison_df['V_Runs'].idxmax()]
        
        print(f"\n💡 QUICK INSIGHTS:")
        print(f"🌟 Best Overall Average: {best_overall_avg['Player']} ({best_overall_avg['O_Avg']})")
        print(f"🏟️ Best Venue Average: {best_venue_avg['Player']} ({best_venue_avg['V_Avg']})")
        print(f"🏆 Most Overall Runs: {most_overall_runs['Player']} ({most_overall_runs['O_Runs']} runs)")
        print(f"🎯 Most Venue Runs: {most_venue_runs['Player']} ({most_venue_runs['V_Runs']} runs)")

# Example usage with comprehensive overall statistics
test_players = [
                "Caleb Jewell", "AHT Donald", "DL Lloyd", "WL Madsen", "SR Patel","B Guest",
        "MK Andersson", "RA Whiteley", "Mohammad Ghazanfar", "Nick Potts", "PR Brown","RS Patel","RH Patel", 
    "SG Budinger", "LPJ Kimber", "Shan Masood", "LJ Hill",
    "OB Cox", "LV van Beek", "L Trevaskis", "TAR Scriven", "MET Salisbury", "RI Walker"]

enhanced_player_analysis_with_overall_stats(df,"Edgbaston, Birmingham", test_players)


🏟️ PLAYER PERFORMANCE ANALYSIS
📍 Venue: Edgbaston, Birmingham

🔍 PLAYER SEARCH RESULTS:
--------------------------------------------------
❌ Caleb Jewell → Not found
   💡 Try: GJ Bell (55.38461538461539%), LC Norwell (54.54545454545454%)
🎯 AHT Donald → AHT Donald (100.0%)
🎯 DL Lloyd → DL Lloyd (100.0%)
🎯 WL Madsen → WL Madsen (100.0%)
🎯 SR Patel → SR Patel (100.0%)
✅ B Guest → BD Guest (93.33333333333333%)
❌ MK Andersson → Not found
   💡 Try: BW Sanderson (66.66666666666667%), MM Ali (54.0%)
🎯 RA Whiteley → RA Whiteley (100.0%)
❌ Mohammad Ghazanfar → Not found
   💡 Try: Mohammad Nabi (72.38095238095238%), Mohammad Amir (72.38095238095238%)
❌ Nick Potts → Not found
   💡 Try: MJ Potts (73.07692307692307%), NA Sowter (42.10526315789473%)
🎯 PR Brown → PR Brown (100.0%)
✅ RS Patel → SR Patel (87.5%)
✅ RH Patel → SR Patel (87.5%)
🎯 SG Budinger → SG Budinger (100.0%)
🎯 LPJ Kimber → LPJ Kimber (100.0%)
🎯 Shan Masood → Shan Masood (100.0%)
🎯 LJ Hill → LJ Hill (100.0%)
🎯 OB Cox → OB Cox (100.0%)

,Player,O_Mat,O_Runs,O_Avg,O_1st,O_2nd,O_BPD,O_Mile,V_Mat,V_Runs,V_Avg,V_SR,V_HS,V_Bnd,V_1st,V_2nd,V_BPD,V_BPB,V_Mile
1,WL Madsen,126,3785,30.0,30.4,29.8,26.74,59|28|2,5,180,36.0,137.4,55,23,50.0,32.5,26.2,5.70,3|2|0
2,SR Patel,124,2627,21.2,23.0,19.5,21.61,34|17|0,4,135,33.8,157.0,64,17,41.0,31.3,21.5,5.06,2|1|0
3,SR Patel,124,2627,21.2,23.0,19.5,21.61,34|17|0,4,135,33.8,157.0,64,17,41.0,31.3,21.5,5.06,2|1|0
4,SR Patel,124,2627,21.2,23.0,19.5,21.61,34|17|0,4,135,33.8,157.0,64,17,41.0,31.3,21.5,5.06,2|1|0
5,AHT Donald,71,1343,18.9,17.4,20.4,14.28,18|9|0,2,60,30.0,157.9,30,9,30.0,30.0,38.0,4.22,2|0|0
6,Shan Masood,40,1014,25.4,30.5,21.1,23.61,17|6|0,3,60,20.0,103.4,45,7,10.0,25.0,29.0,8.29,1|0|0
7,RA Whiteley,137,2518,18.4,18.0,18.9,17.11,28|5|0,7,122,17.4,119.6,28,12,18.8,15.7,17.0,8.50,0|0|0
8,DL Lloyd,86,1962,22.8,28.0,18.1,18.70,27|12|0,2,24,12.0,184.6,13,4,0.0,12.0,6.5,3.25,0|0|0
9,BD Guest,50,810,16.2,15.7,16.6,22.57,10|1|0,5,59,11.8,147.5,20,5,0.0,11.8,40.0,8.00,0|0|0
10,OB Cox,125,2363,18.9,20.6,17.1,22.99,28|8|0,4,47,11.8,111.9,16,4,10.0,12.3,10.5,10.50,0|0|0



💡 QUICK INSIGHTS:
🌟 Best Overall Average: WL Madsen (30.0)
🏟️ Best Venue Average: WL Madsen (36.0)
🏆 Most Overall Runs: WL Madsen (3785 runs)
🎯 Most Venue Runs: WL Madsen (180 runs)
